In [140]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import ast
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [91]:
df = pd.read_csv('HAM10000_metadata')
my_array = np.load('array_col.npy', allow_pickle = True)
s = pd.Series(my_array, name = 'image')
df = pd.concat([df, s], axis=1)

In [92]:
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"[[[191, 152, 192], [194, 154, 193], [196, 152,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"[[[24, 14, 29], [56, 39, 64], [106, 80, 112], ..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"[[[190, 136, 150], [199, 145, 161], [200, 141,..."


In [6]:
type(df['image'][0])

numpy.ndarray

In [8]:
df['image'][0].reshape(-1, 1)

array([[191],
       [152],
       [192],
       [194],
       [154],
       [193],
       [196],
       [152],
       [184],
       [199],
       [158],
       [189],
       [205],
       [165],
       [196],
       [209],
       [166],
       [205],
       [208],
       [164],
       [201],
       [208],
       [166],
       [195],
       [213],
       [169],
       [199],
       [215],
       [178],
       [203],
       [216],
       [181],
       [205],
       [215],
       [174],
       [206],
       [215],
       [172],
       [205],
       [217],
       [172],
       [210],
       [216],
       [163],
       [198],
       [215],
       [163],
       [205],
       [216],
       [170],
       [207],
       [218],
       [172],
       [204],
       [216],
       [167],
       [198],
       [213],
       [163],
       [190],
       [208],
       [161],
       [187],
       [202],
       [150],
       [171],
       [205],
       [155],
       [186],
       [205],
       [160],
      

In [93]:
def reshape(lst):
    lst = lst.reshape(-1, 1)
    return lst

In [94]:
df['image'] = df['image'].apply(reshape)

In [95]:
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"[[191], [152], [192], [194], [154], [193], [19..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"[[24], [14], [29], [56], [39], [64], [106], [8..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"[[190], [136], [150], [199], [145], [161], [20..."


In [96]:
le = LabelEncoder()
le.fit(df['dx'])
LabelEncoder()
df['label'] = le.transform(df["dx"]) 
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"[[191], [152], [192], [194], [154], [193], [19...",2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"[[24], [14], [29], [56], [39], [64], [106], [8...",2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"[[190], [136], [150], [199], [145], [161], [20...",2


In [97]:
def nest_flatten(lst):
    flat_list = [num for sublist in lst for num in sublist]
    return flat_list

In [98]:
df['image'] = df['image'].apply(nest_flatten)

In [99]:
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"[191, 152, 192, 194, 154, 193, 196, 152, 184, ...",2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"[24, 14, 29, 56, 39, 64, 106, 80, 112, 143, 11...",2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"[190, 136, 150, 199, 145, 161, 200, 141, 160, ...",2


In [103]:
len(df['image'][8888])

3072

In [104]:
def further_flatten(lst):
    output_string = ','.join(str(i) for i in lst)
    return output_string

In [105]:
df['image'] = df['image'].apply(further_flatten)

In [106]:
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image,label
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"191,152,192,194,154,193,196,152,184,199,158,18...",2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"24,14,29,56,39,64,106,80,112,143,112,144,167,1...",2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"190,136,150,199,145,161,200,141,160,205,148,16...",2


In [115]:
X = df['image']
y = df['label']

In [119]:
image_list = df['image'].str.split(',').tolist()

In [120]:
for i in range(len(image_list[0])):
    col_name = f'col{i+1}'
    df[col_name] = [int(x[i]) for x in image_list]

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

...

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = [int(x[i]) for x in image_list]
C:\Users\beelk\AppData\Local\Temp\ipykernel_17324\886685394.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [121]:
df.head(3)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,image,label,...,col3063,col3064,col3065,col3066,col3067,col3068,col3069,col3070,col3071,col3072
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,"191,152,192,194,154,193,196,152,184,199,158,18...",2,...,136,176,131,146,184,149,169,184,153,176
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,"24,14,29,56,39,64,106,80,112,143,112,144,167,1...",2,...,69,43,26,43,25,14,28,25,14,27
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,"190,136,150,199,145,161,200,141,160,205,148,16...",2,...,143,165,129,144,155,121,138,134,102,114


In [122]:
X = df.drop(columns = ['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization', 'dataset', 'image', 'label'])

In [123]:
X

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col3063,col3064,col3065,col3066,col3067,col3068,col3069,col3070,col3071,col3072
0,191,152,192,194,154,193,196,152,184,199,...,136,176,131,146,184,149,169,184,153,176
1,24,14,29,56,39,64,106,80,112,143,...,69,43,26,43,25,14,28,25,14,27
2,190,136,150,199,145,161,200,141,160,205,...,143,165,129,144,155,121,138,134,102,114
3,35,18,28,83,51,64,128,88,103,161,...,56,34,18,26,25,12,16,25,12,15
4,155,110,135,188,139,168,210,157,184,220,...,196,200,158,175,160,124,137,103,73,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,181,164,179,179,162,176,180,163,178,183,...,186,207,184,185,207,186,186,205,187,188
10011,4,5,3,24,22,21,101,88,88,128,...,108,70,59,57,13,13,11,4,5,3
10012,132,119,120,157,139,138,177,158,162,174,...,177,206,184,181,171,148,142,176,156,151
10013,160,123,144,163,131,152,166,128,148,168,...,165,184,159,167,185,158,167,184,161,170


In [124]:
y = df['label']

In [125]:
y

0        2
1        2
2        2
3        2
4        2
        ..
10010    0
10011    0
10012    0
10013    0
10014    4
Name: label, Length: 10015, dtype: int32

In [128]:
smote = SMOTE()
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

2    6705
5    6705
3    6705
4    6705
6    6705
1    6705
0    6705
Name: label, dtype: int64

In [129]:
X_sm

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col3063,col3064,col3065,col3066,col3067,col3068,col3069,col3070,col3071,col3072
0,191,152,192,194,154,193,196,152,184,199,...,136,176,131,146,184,149,169,184,153,176
1,24,14,29,56,39,64,106,80,112,143,...,69,43,26,43,25,14,28,25,14,27
2,190,136,150,199,145,161,200,141,160,205,...,143,165,129,144,155,121,138,134,102,114
3,35,18,28,83,51,64,128,88,103,161,...,56,34,18,26,25,12,16,25,12,15
4,155,110,135,188,139,168,210,157,184,220,...,196,200,158,175,160,124,137,103,73,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46930,190,159,180,191,160,181,192,161,183,192,...,180,194,158,180,193,158,180,194,161,182
46931,208,141,138,211,141,137,213,146,142,214,...,134,196,137,132,194,136,131,194,137,131
46932,203,141,156,204,140,157,205,140,161,208,...,155,196,146,154,194,144,152,191,140,148
46933,218,156,176,220,159,178,221,157,178,222,...,168,213,163,168,213,162,168,211,160,169


In [130]:
X_sm = X_sm/255.

In [134]:
X_sm.to_csv('X_sm.csv')

In [135]:
y_sm.to_csv('y_sm.csv')

In [139]:
X_sm

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col3063,col3064,col3065,col3066,col3067,col3068,col3069,col3070,col3071,col3072
0,0.749020,0.596078,0.752941,0.760784,0.603922,0.756863,0.768627,0.596078,0.721569,0.780392,...,0.533333,0.690196,0.513725,0.572549,0.721569,0.584314,0.662745,0.721569,0.600000,0.690196
1,0.094118,0.054902,0.113725,0.219608,0.152941,0.250980,0.415686,0.313725,0.439216,0.560784,...,0.270588,0.168627,0.101961,0.168627,0.098039,0.054902,0.109804,0.098039,0.054902,0.105882
2,0.745098,0.533333,0.588235,0.780392,0.568627,0.631373,0.784314,0.552941,0.627451,0.803922,...,0.560784,0.647059,0.505882,0.564706,0.607843,0.474510,0.541176,0.525490,0.400000,0.447059
3,0.137255,0.070588,0.109804,0.325490,0.200000,0.250980,0.501961,0.345098,0.403922,0.631373,...,0.219608,0.133333,0.070588,0.101961,0.098039,0.047059,0.062745,0.098039,0.047059,0.058824
4,0.607843,0.431373,0.529412,0.737255,0.545098,0.658824,0.823529,0.615686,0.721569,0.862745,...,0.768627,0.784314,0.619608,0.686275,0.627451,0.486275,0.537255,0.403922,0.286275,0.341176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46930,0.745098,0.623529,0.705882,0.749020,0.627451,0.709804,0.752941,0.631373,0.717647,0.752941,...,0.705882,0.760784,0.619608,0.705882,0.756863,0.619608,0.705882,0.760784,0.631373,0.713725
46931,0.815686,0.552941,0.541176,0.827451,0.552941,0.537255,0.835294,0.572549,0.556863,0.839216,...,0.525490,0.768627,0.537255,0.517647,0.760784,0.533333,0.513725,0.760784,0.537255,0.513725
46932,0.796078,0.552941,0.611765,0.800000,0.549020,0.615686,0.803922,0.549020,0.631373,0.815686,...,0.607843,0.768627,0.572549,0.603922,0.760784,0.564706,0.596078,0.749020,0.549020,0.580392
46933,0.854902,0.611765,0.690196,0.862745,0.623529,0.698039,0.866667,0.615686,0.698039,0.870588,...,0.658824,0.835294,0.639216,0.658824,0.835294,0.635294,0.658824,0.827451,0.627451,0.662745


In [141]:
y_sm = to_categorical(y_sm, num_classes=7)

In [145]:
y_sm = pd.DataFrame(y_sm)

In [146]:
y_sm.head(3)

,0,1,2,3,4,5,6
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
